In [1]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("20"))

puzzle.header()

# Race Condition

[Open Website](https://adventofcode.com/2024/day/20)

In [2]:
# helper functions
def domain_from_input(input: InputReader) -> (list, list, list):
    matrix = input.matrix()

    # find start
    start = None
    for x, y, value in matrix:
        if value == 'S':
            start = (x, y)
            break

    # find end
    end = None
    for x, y, value in matrix:
        if value == 'E':
            end = (x, y)
            break

    return matrix, start, end


test_input, _, _ = domain_from_input(puzzle.example(0))
test_input.print()

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############


In [8]:

# test case (part 1)
from common.matrix import Matrix, MatrixNavigator, Direction
from collections import deque

directions = [
    Direction.UP,
    Direction.DOWN,
    Direction.LEFT,
    Direction.RIGHT
]


def available_options(matrix: Matrix, point: tuple, with_cheat: bool, cache: dict) -> list:
    key = (point, with_cheat)
    if key in cache:
        return cache[key]

    pointer = MatrixNavigator(matrix, point[0], point[1])
    options = []
    for direction in directions:
        ok, value = pointer.peek_value(direction)
        if ok:
            if with_cheat or value != '#':
                new_pointer = pointer.copy()
                new_pointer.move(direction)
                options.append(new_pointer.get_position())

    cache[key] = options
    return options


def find_shortest_path(matrix: Matrix, start: tuple, end: tuple, cache: dict) -> list:
    if start in cache:
        return cache[start]

    queue = deque([(start, [start])])
    visited = set()
    visited.add(start)

    while queue:
        current_point, path = queue.popleft()
        if current_point == end:
            cache[start] = path
            return path

        for next_point in available_options(matrix, current_point, False, cache):
            if next_point not in visited:
                visited.add(next_point)
                queue.append((next_point, path + [next_point]))

    return []  # Return an empty list if no path is found


def find_shortest_path_count(matrix: Matrix, start: tuple, end: tuple, cache: dict) -> int:
    if start in cache:
        return cache[start]

    queue = deque([(start, 0)])
    visited = set()
    visited.add(start)

    while queue:
        current_point, path_length = queue.popleft()
        if current_point == end:
            cache[start] = path_length
            return path_length

        for next_point in available_options(matrix, current_point, False, cache):
            if next_point not in visited:
                visited.add(next_point)
                queue.append((next_point, path_length + 1))

    return -1  # Return -1 if no path is found


def part_1(reader: InputReader, min_savings: int, debug: bool) -> int:
    path_cache = {}
    path_count_cache = {}
    options_cache = {}

    matrix, start, end = domain_from_input(reader)
    initial_solution = find_shortest_path(matrix, start, end, path_cache)
    initial_solution_length = len(initial_solution)

    solutions = {}

    for point in initial_solution:
        new_starts = []
        steps = initial_solution.index(point)

        # find all "." within two steps
        for point_1 in available_options(matrix, point, True, options_cache):
            for point_2 in available_options(matrix, point_1, False, options_cache):
                new_starts.append(point_2)

        steps += 2

        for new_start in new_starts:
            # if there's a solution
            solution_length = find_shortest_path_count(matrix, new_start, end, path_count_cache)
            if solution_length > -1:
                new_total = solution_length + steps
                if new_total < initial_solution_length:
                    solutions[new_total] = solutions.get(new_total, 0) + 1

    total = 0

    if debug:
        matrix.print()
        print()

    for key in sorted(solutions.keys(), reverse=True):
        savings = initial_solution_length - key
        if savings >= min_savings:
            total += solutions[key]
        if debug:
            print(f"There are {solutions[key]} cheats that save {savings} picoseconds")

    if debug:
        print()

    return total


result = part_1(puzzle.example(0), 20, True)
display(result)
assert result == 5

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############

There are 116 cheats that save 1 picoseconds
There are 14 cheats that save 3 picoseconds
There are 14 cheats that save 5 picoseconds
There are 2 cheats that save 7 picoseconds
There are 4 cheats that save 9 picoseconds
There are 2 cheats that save 11 picoseconds
There are 3 cheats that save 13 picoseconds
There are 1 cheats that save 21 picoseconds
There are 1 cheats that save 37 picoseconds
There are 1 cheats that save 39 picoseconds
There are 1 cheats that save 41 picoseconds
There are 1 cheats that save 65 picoseconds



5

In [9]:
# real case (part 1)
result = part_1(puzzle.input(), 100, False)
display(result)

1497

In [ ]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    lines = domain_from_input(reader)
    if debug:
        display(lines)
    return 0


result = part_2(puzzle.example(0), True)
display(result)
assert result == 0

In [ ]:
# real case (part 2)
result = part_2(puzzle.input(), False)
display(result)

In [ ]:
# print easters eggs
puzzle.print_easter_eggs()